# 12-Finding Protein Folding by Quantum Annealing

量子アニーリングを用いたタンパク質折り畳み問題がハーバード大学の先生によって2012年に発表されました。

natureに掲載されているの論文をベースにします。

Finding low-energy conformations of lattice protein models by quantum annealing Alejandro Perdomo-Ortiz, Neil Dickson, Marshall Drew-Brook, Geordie Rose & Alán Aspuru-Guzik Scientific Reports volume 2, Article number: 571 (2012)
- https://www.nature.com/articles/srep00571

## 問題の概要とHPモデル、Mijazawa-Jernigan(MJ)モデル 

### 問題概要

- タンパク質の立体構造をラティスフォールディングモデル(https://en.wikipedia.org/wiki/Lattice_protein)という単純な格子モデルとして捉え、HUBOの形式にエンコードし、解くことです。
- 特に参考論文では、２次元上の構造問題として見做し、解いています。
- ラティスフォールディングモデルとして、タンパク質の構造を予測するために、タンパク質に含まれるアミノ酸の疎水性のみを用いて、タンパク質の構造を予測するHPモデルを使用しています。

<div>
    <img src="images/hp_model1.png" width="300" height="200">
</div>

### HPモデル 
- アミノ酸をＨ（疎水性、非極性アミノ酸）とＰ(親水性、極性アミノ酸）のいずれかに分ける。
- 参考：http://www.iba.t.u-tokyo.ac.jp/iba/AI/HP.pdf

### MJモデル
- 今回は単純化されたMJモデルを利用します。

## MJモデルのHUBOへの適用

用意された塩基列を特定の方向に回転させる操作をイジングモデルに対応させています。

今回使用する塩基列は、PSVKMAの配列で下記のように特定の塩基列が隣接すると安定状態になり、エネルギーが減ります。

このエネルギーの安定化を使っってコスト関数を最小化させることを考えます。また、今回塩基列を全て一度に処理するのは難しいのでいくつかのパターンに分けます。

すでにいくつかの折り畳まれたパターンから出発して安定状態を求めます。
数が多くないので書き出すことができ、最も低いエネルギー安定状態を求めることができます。

本チュートリアルでは、実験３のschemeに絞り、取り上げて一番エネルギーの低い状態を立式から導出したいと思います。

## 　コスト関数

タンパク質の構造をラティスフォールディングモデルとして考えるにあたり、一般のコスト関数は、以下となります。

$$E_p = E_{onsite} + E_{pw} + E_{ext}$$

- 第１項: タンパク質の塩基列が重ならない
- 第２項: は塩基列同士の近接の相互作用のエネルギー
- 第３項: 外部からの影響

論文中の実験３については、第３項は使用しないため、コスト関数は、以下となります。

$$E_p = E_{onsite} + E_{pw}$$


## モデル

今回、論文中にも触れられている実験３をやって見たいと思います。塩基列は、

P-S-V-K-M-A

この順番ですが、今回はある程度折りたたまれた状態で始まります。


<div>
    <img src="images/scheme.png" width="600" height="600">
</div>

この場合、PSKVまでは決まっていて、Mは下か左が決まっていますので、回転方向はPから順番に書いてみると、

$$010010q_10q_2q_3$$

となり、３量子ビットの式を解けばよいことになります。$01$は右、$00$は下、$10$は左、Mは下か左しかないので、$00$か$10$なので、$0$は決まっていて、残りの$q_1$から$q_3$をイジング問題で求めます。

<div>
    <img src="images/make_costfunction_about_protein_folding.png" width="600" height="400">
</div>


コスト関数は、

$$E = -1-4q_3+9q_1q_3+9q_2q_3-16q_1q_2q_3$$